In [1]:
import pandas as pd
import numpy as np

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [4]:
class finominaStock():
    def __init__(self):
        self.chrome_options = webdriver.ChromeOptions()
        self.driver = webdriver.Chrome(executable_path = "./webdriver/chromedriver.exe",chrome_options=self.chrome_options)
        self.driver.set_page_load_timeout(60)
        self.start_symbol = ""
        
    def run(self):
        if self.start_symbol == "":
            start = True
        else:
            start = False
        ####################################
        df = pd.read_excel("stockList.xlsx")
        for i,r in df.iterrows():
            if r['Symbol'] == self.start_symbol:
                start = True
            
            if start :
                print(f"{r['Symbol']} [running]",end="\r")
                try:
                    self.driver.get(f"https://www.finnomena.com/stock/{r['Symbol']}")
                    WebDriverWait(self.driver, 0,10).until(EC.visibility_of_element_located((By.ID, "Close")))
                except:
                    print(f"{r['Symbol']} [error]   ")
                growthrate = self.financial_ratio("stock-growthrate")
                ratio = self.financial_ratio("stock-financial-ratio")
                statistics = self.financial_ratio("stock-statistics")
                #financial_report = self.financial_ratio("stock-financial-report")
                
               

                data = pd.concat([growthrate, ratio, statistics],axis=1, join='outer')
                data.drop(data.tail(1).index,inplace=True) 
                
                data['Symbol'] = r['Symbol']
                data['Market'] = r['Market']
                data['IndusGroup'] = r['IndusGroup']
                data['BusinessGroup'] = r['BusinessGroup']
                
                data.to_csv(f"./data/RWST/{r['Symbol']}.csv",mode='w',encoding='utf-8')
                print(f"{r['Symbol']} [done]   ")
            
    
    #financial-ratio
    def financial_ratio( self,Id ):
        topic = self.driver.find_element(By.ID, Id).find_element(By.CLASS_NAME, 'topic-wrapper').text.split("\n")
        del topic[0]

        content = self.driver.find_element(By.ID, Id).find_element(By.CLASS_NAME, 'content')
        date = content.find_element(By.CLASS_NAME, 'timeline-wrapper').text

        data = {}
        for i,r in enumerate( content.find_elements(By.CLASS_NAME,"data-wrapper") ):
            data[f"{topic[i]}"] = []
            for c in r.find_elements(By.CLASS_NAME,"data-each"):
                data[f"{topic[i]}"] += [c.text]

        data = pd.DataFrame(data,index=date.split("\n"))
        #data["Date"] = date.split("\n")
        return data

In [6]:
finomina = finominaStock()
finomina.start_symbol = ''
finomina.run()

COM7 [error]   
COM7 [done]   
